In [177]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.linear_model  import Ridge,Lasso,RidgeCV, LassoCV, ElasticNet, ElasticNetCV, LogisticRegression
import pickle

In [178]:
df = pd.read_excel(r'D:\DS\DS-CSFB\MODRED_AUDIT_SURAT_practise.xlsx')
df.head()

,MRBTS,LNBTS,MODPR,MODRED,id,addUtranSIToRedirMsg,csFallBPrio,emerCallPrio,redirFreqEutra,redirFreqUtra,redirRAT,redirectPrio,addGsmSIToRedirMsg,redirGeranArfcnStructL,Item-redirGeranArfcnStructL-redirGeranArfcnPrio,Item-redirGeranArfcnStructL-redirGeranArfcnValue,redirGeranBandIndicator,redirFreqCdma,redirFreqUtraTDDL,Remarks
0,708330,708330,0,0,NaN,2.0,1,1,41140,10807.0,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error
1,711566,711566,0,1,NaN,NaN,1,1,41140,NaN,1,1,2.0,List,1;2;3;4;5;6;7;8;9,1;2;49;50;51;52;53;54;55,0.0,NaN,NaN,OK
2,801600,801600,0,0,NaN,2.0,2,2,1309,10807.0,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OK
3,711561,711561,0,1,NaN,NaN,1,1,41140,NaN,1,1,2.0,List,1;2;3;4;5;6;7;8;9,1;2;49;50;51;52;53;54;55,0.0,NaN,NaN,OK
4,711605,711605,0,1,NaN,NaN,1,1,41140,NaN,1,1,2.0,List,1;2;3;4;5;6;7;8;9;10,26;27;28;29;30;31;32;33;34;35,0.0,NaN,NaN,OK


In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5123 entries, 0 to 5122
Data columns (total 20 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0          MRBTS                                             5123 non-null   int64  
 1          LNBTS                                             5123 non-null   int64  
 2          MODPR                                             5123 non-null   int64  
 3          MODRED                                            5123 non-null   int64  
 4   id                                                       0 non-null      float64
 5          addUtranSIToRedirMsg                              2625 non-null   float64
 6          csFallBPrio                                       5123 non-null   int64  
 7          emerCallPrio                                      5123 non-null   int64  
 8          redirFreqEutra       

In [180]:
df.columns = df.columns.str.replace(' ','')
df.head()

,MRBTS,LNBTS,MODPR,MODRED,id,addUtranSIToRedirMsg,csFallBPrio,emerCallPrio,redirFreqEutra,redirFreqUtra,redirRAT,redirectPrio,addGsmSIToRedirMsg,redirGeranArfcnStructL,Item-redirGeranArfcnStructL-redirGeranArfcnPrio,Item-redirGeranArfcnStructL-redirGeranArfcnValue,redirGeranBandIndicator,redirFreqCdma,redirFreqUtraTDDL,Remarks
0,708330,708330,0,0,NaN,2.0,1,1,41140,10807.0,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Error
1,711566,711566,0,1,NaN,NaN,1,1,41140,NaN,1,1,2.0,List,1;2;3;4;5;6;7;8;9,1;2;49;50;51;52;53;54;55,0.0,NaN,NaN,OK
2,801600,801600,0,0,NaN,2.0,2,2,1309,10807.0,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OK
3,711561,711561,0,1,NaN,NaN,1,1,41140,NaN,1,1,2.0,List,1;2;3;4;5;6;7;8;9,1;2;49;50;51;52;53;54;55,0.0,NaN,NaN,OK
4,711605,711605,0,1,NaN,NaN,1,1,41140,NaN,1,1,2.0,List,1;2;3;4;5;6;7;8;9;10,26;27;28;29;30;31;32;33;34;35,0.0,NaN,NaN,OK


In [181]:
df.shape

(5123, 20)

In [182]:
class Preprocessor:

    def remove_columns(self,data,columns):
        self.data=data
        self.columns=columns
        try:
            self.useful_data=self.data.drop(self.columns, axis=1) # drop the labels specified in the columns
            print('Column removal Successful.Exited the remove_columns method of the Preprocessor class')
            return self.useful_data
        except Exception as e:
            return print('Error %s' % e)
            raise Exception()

    def handle_missing_values(self, data):
        self.data= data
        try:
            self.data.fillna(0, inplace=True)
            print('Missing values replace with 0 in all columns')
            return self.data
        except Exception as e:
            print('Exception message:  ' + str(e))
            raise Exception()
    
    def handle_categorical(self, data):
        self.data = data
        try:
            cleanup = {'OK' : 1, 'Error' : 0}
            self.data.replace(cleanup, inplace= True)
            return self.data
            print('categorical data is handling done')
        except Exception as e:
            print('Exception message % s' % e)
            raise Exception()

In [183]:
columns= ['id', 'MRBTS','MODPR','redirGeranArfcnStructL','Item-redirGeranArfcnStructL-redirGeranArfcnPrio', 'Item-redirGeranArfcnStructL-redirGeranArfcnValue','redirFreqCdma','redirFreqUtraTDDL','redirGeranBandIndicator']

In [184]:
a1= Preprocessor()

In [185]:
df2 = a1.handle_missing_values(df)

Missing values replace with 0 in all columns


In [186]:
df2.isnull().sum()

MRBTS                                               0
LNBTS                                               0
MODPR                                               0
MODRED                                              0
id                                                  0
addUtranSIToRedirMsg                                0
csFallBPrio                                         0
emerCallPrio                                        0
redirFreqEutra                                      0
redirFreqUtra                                       0
redirRAT                                            0
redirectPrio                                        0
addGsmSIToRedirMsg                                  0
redirGeranArfcnStructL                              0
Item-redirGeranArfcnStructL-redirGeranArfcnPrio     0
Item-redirGeranArfcnStructL-redirGeranArfcnValue    0
redirGeranBandIndicator                             0
redirFreqCdma                                       0
redirFreqUtraTDDL           

In [187]:
df3 = a1.handle_categorical(df2)

In [188]:
df3['Remarks'].head()

0    0
1    1
2    1
3    1
4    1
Name: Remarks, dtype: int64

In [189]:
print(type(df))
print(type(df2))
print(type(df3))
print(df.shape)
print(df2.shape)
print(df3.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
(5123, 20)
(5123, 20)
(5123, 20)


In [190]:
df4= a1.remove_columns(df3,columns)

Column removal Successful.Exited the remove_columns method of the Preprocessor class


In [191]:
df4.head()

,LNBTS,MODRED,addUtranSIToRedirMsg,csFallBPrio,emerCallPrio,redirFreqEutra,redirFreqUtra,redirRAT,redirectPrio,addGsmSIToRedirMsg,Remarks
0,708330,0,2.0,1,1,41140,10807.0,2,1,0.0,0
1,711566,1,0.0,1,1,41140,0.0,1,1,2.0,1
2,801600,0,2.0,2,2,1309,10807.0,2,2,0.0,1
3,711561,1,0.0,1,1,41140,0.0,1,1,2.0,1
4,711605,1,0.0,1,1,41140,0.0,1,1,2.0,1


In [192]:
df4.shape

(5123, 11)

In [193]:
x = df4.drop(columns = 'Remarks')
x.head()

,LNBTS,MODRED,addUtranSIToRedirMsg,csFallBPrio,emerCallPrio,redirFreqEutra,redirFreqUtra,redirRAT,redirectPrio,addGsmSIToRedirMsg
0,708330,0,2.0,1,1,41140,10807.0,2,1,0.0
1,711566,1,0.0,1,1,41140,0.0,1,1,2.0
2,801600,0,2.0,2,2,1309,10807.0,2,2,0.0
3,711561,1,0.0,1,1,41140,0.0,1,1,2.0
4,711605,1,0.0,1,1,41140,0.0,1,1,2.0


In [194]:
x.shape

(5123, 10)

In [195]:
y = df4['Remarks']
y.head()

0    0
1    1
2    1
3    1
4    1
Name: Remarks, dtype: int64

In [198]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.40,random_state=100)
log_reg = LogisticRegression()
log_reg.fit(x,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [199]:
list(y_train)

[1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [200]:
y_pred = log_reg.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
accuracy

0.9556097560975609

In [202]:
Des_tree1 = DecisionTreeClassifier(min_samples_leaf=3, random_state=5)
Des_tree1.fit(x, y)
Des_tree1.score(x_test,y_test)

0.9819512195121951

In [208]:
import pickle
with open( 'DTmodel_Pred_MODREDV5.sav', 'wb') as f:
    pickle.dump(clf,f)

In [206]:
from sklearn.ensemble import RandomForestClassifier 
clf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=100,
                       verbose=0, warm_start=False)
clf.fit(x_train, y_train) 
  


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=100,
                       verbose=0, warm_start=False)

In [207]:
# performing predictions on the test dataset 
y_pred = clf.predict(x_test) 
  
# metrics are used to find accuracy or error 
from sklearn import metrics   
print() 
  
# using metrics module for accuracy calculation 
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))


ACCURACY OF THE MODEL:  0.9975609756097561
